# Line Integrated Diagnostics

[LineIntegratedDiagnostic]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic.rst#plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic

Many plasma diagnostics use a beam of light or particles to probe the properties of a plasma. Such measurements are inherently line-integrated: for example, an imaging interferometer reduces the density along the probe beam path, $n(x,y,z)$, to a 2D plane of phase shifts, $\Delta \phi(x,y)$. 

The [LineIntegratedDiagnostic] class in PlasmaPy provides an abstract framework for creating synthetic diagnostics that work in this manner. These synthetic diagnostics can then be used to predict the results of experimental measurements given an analytical model or simulation result describing a plasma. In this notebook, we will review the synthetic diagnostics implemented in PlasmaPy as subclasses of LineIntegrated Diagnostic. We will then explicitly re-create the `Interferometer` synthetic diagnostic class to provide an example of how the [LineIntegratedDiagnostic] abstract class can be subclassed to create a custom synthetic diagnostic.

In [ ]:
import astropy.constants as const
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from scipy.signal import medfilt

from plasmapy.diagnostics.path_integrated_diagnostic import (
    Interferometer,
    LineIntegratedDiagnostic,
)
from plasmapy.plasma.grids import CartesianGrid

## Contents

1. [The Line Integrated Diagnostic Framework](#The-Line-Integrated-Diagnostic-Framework)
1. [Interferometer](#Interferometer)
1. [Creating a Custom Diagnostic Class](#Creating-a-Custom-Diagnostic-Class)


## The Line Integrated Diagnostic Framework

[LineIntegratedDiagnostic]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic.rst#plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic

[Grid]: ../../api/plasmapy.plasma.grids.rst#plasmapy.plasma.grids

<img src="line_integrated_diagnostic_setup.png">

A [LineIntegratedDiagnostic] class object is instantiated by providing a [Grid] object (representing the plasma to be measured) as well as vectors from the origin of the [Grid] to the detector plane and the source plane. The line integral through the grid is then calculated using the `line_integral` method. This method takes keywords that define the size and resolution of the detector. The line integral is then calculated using the following method:

1. A 2D array of points (Nx, Ny) in the detector plane is created. A corresponding array of points is created in the source plane. If the `collimated` keyword is False, all of the source points will instead be set equal to the point defined by the source vector. 
2. Equations are calculated for lines that connect each point in the detector plane to the corresponding point in the source plane. 
3. A number of points (set by the `num` keyword) are generated on each line in the region where the lines cross the grid. The result is a 3D array with dimensions (Nx, Ny, num).
4. The `integrand()` method is evaluated at each point in the 3D array.
5. The resulting integrand array is then integrated along the last dimension to produce the final (Nx,Ny) array representing the line-integral as collected in the detector plane.

Subclasses of [LineIntegratedDiagnostic] overwrite the `integrand()` method in order to line-integrate different quantities and thus represent different diagnostics.

## Interferometer

[Interferometer]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.interferometry.Interferometer.rst#plasmapy.diagnostics.path_integrated_diagnostic.interferometry.Interferometer

An interferometer diagnoses plasma density by measuring the phase shift ($\Delta \phi$) of a probe beam. The phase shift is 

\begin{equation}
\Delta \phi = -\frac{\omega_{probe}}{2 c n_c} \int n_e dl
\end{equation}

Where $\omega_{probe}$ is the probe beam frequency, $c$ is the speed of light, $\int n_e dl$ is the line-integrated electron density, and $n_c$ is the critical density

\begin{equation}
n_c = \frac{\epsilon_0 m_e}{e^2} \omega_{probe}^2
\end{equation}

The PlasmaPy [Interferometer] class calculates $\Delta \phi$ given a grid of electron densities. In order to demonstrate the use of this class, we will first create a cylinder of constant electron density.

In [ ]:
axis = np.linspace(-2, 2, num=200) * u.mm
xarr, yarr, zarr = np.meshgrid(axis, axis, axis, indexing="ij")
r = np.sqrt(xarr ** 2 + yarr ** 2)
r0 = 1 * u.mm
n_e = np.where(r < r0, np.cos((r / r0).value * np.pi / 2), 0) * 2e19 / u.cm ** 3
grid = CartesianGrid(xarr, yarr, zarr)
grid.add_quantities(n_e=n_e)


fig, ax = plt.subplots(ncols=2, figsize=(12, 6))
fig.subplots_adjust(right=1)
p1 = ax[0].pcolormesh(
    axis.to(u.mm).value, axis.to(u.mm).value, grid["n_e"][:, :, 100], shading="auto"
)
ax[0].set_xlabel("X (mm)")
ax[0].set_ylabel("Y (mm)")
ax[0].set_aspect("equal")

p2 = ax[1].pcolormesh(
    axis.to(u.mm).value, axis.to(u.mm).value, grid["n_e"][:, 100, :], shading="auto"
)
ax[1].set_xlabel("X (mm)")
ax[1].set_ylabel("Z (mm)")
ax[1].set_aspect("equal")

cb = fig.colorbar(p2, ax=ax, orientation="vertical", pad=0.03, shrink=0.7)
cb.ax.set_ylabel("Density (cm$^{-3}$)", fontsize=16);

Next we will instantiate an Interferometer object with this grid

In [ ]:
source = (0 * u.mm, -5 * u.mm, 0 * u.mm)
detector = (0 * u.mm, 5 * u.mm, 0 * u.mm)
obj = Interferometer(grid, source, detector, verbose=False)

Finally, we will use the `evaluate` function to carry out the line integration and calculate the phase shift. This function takes the following arguments

- $\omega_{probe}$, the frequency of the probe beam in Hz
- `num` : The number of interpolated points over which the line integral is carried out.

and the keywords
- `size` : An array defining the size of the detector plane
- `bins` : A list of two values defining the horizontal and vertical resolution of the detector plane.
- `unwrapped` : If true, returns the total phase shift (without 2$\pi$ discontinuities).

In [ ]:
wprobe = 1.14e15 * u.Hz
size = np.array([[-1, 1], [-1, 1]]) * 1.2 * u.mm
bins = [100, 100]
hax, vax, phase = obj.evaluate(wprobe, 30, size=size, bins=bins, unwrapped=False)
hax = hax.to(u.mm).value
vax = vax.to(u.mm).value

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot = ax.pcolormesh(hax, vax, phase.T / np.pi, cmap="binary", shading="auto")
ax.set_aspect("equal")
ax.set_xlabel("X (mm)", fontsize=16)
ax.set_ylabel("Z (mm)", fontsize=16)

cb = fig.colorbar(plot, ax=ax, orientation="vertical", pad=0.03, shrink=0.7)
cb.ax.set_ylabel("Phase Shift / $\pi$", fontsize=16);

The resulting image represents the line-integral through the 3D grid, projected onto the detector plane. Running the `evaluate` method with more `bins` and larger `num` will create a higher resolution image.

In [ ]:
size = np.array([[-1, 1], [-1, 1]]) * 1.2 * u.mm
bins = [300, 300]
hax, vax, phase = obj.evaluate(wprobe, 400, size=size, bins=bins, unwrapped=False)
hax = hax.to(u.mm).value
vax = vax.to(u.mm).value

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot = ax.pcolormesh(hax, vax, phase.T / np.pi, cmap="binary", shading="auto")
ax.set_aspect("equal")
ax.set_xlabel("X (mm)", fontsize=16)
ax.set_ylabel("Z (mm)", fontsize=16)

cb = fig.colorbar(plot, ax=ax, orientation="vertical", pad=0.03, shrink=0.7)
cb.ax.set_ylabel("Phase Shift / $\pi$", fontsize=16);

[scipy.signal.medfilt]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.medfilt.html

The alternating fringes in the image are phase jumps which occur when the phase shift exceeds $2 \pi$. With an experimental interferogram, these phase jumps must be 'unwrapped' to recover the total phase shift, which is possible when a region of zero phase shift is present in the image. In this case, $\Delta \phi = 0$ along the horizontal boundaries of the image, so each row of the interferogram can be unwrapped separately.

Since this method is sensitive to the finite resolution noise on the interferogram, we will first average over the vertical dimension and apply a median filter ([scipy.signal.medfilt]) to smooth the profile.

In [ ]:
mean_phase = np.mean(phase, axis=1)
filt_phase = medfilt(mean_phase, 5)
unwrapped = np.unwrap(filt_phase)

fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(hax, mean_phase, label="Mean")
ax.plot(hax, filt_phase, label="Filtered")
ax.plot(hax, unwrapped, label="Unwrapped")
ax.legend(fontsize=12)
ax.set_xlabel("X (mm)", fontsize=16)
ax.set_ylabel("$\Delta \phi$", fontsize=16);

The line-integrated density can then be recovered from the unwrapped phase

\begin{equation}
\int n_e dl = \frac{2 c \epsilon_0 m_e}{e^2} \omega_p \Delta \phi
\end{equation}

In [ ]:
int_ne = (
    -2 * const.c * const.eps0.si * const.m_e / const.e.si ** 2 * wprobe * unwrapped
).to(u.cm ** -2)

fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(hax, int_ne.value)
ax.set_xlabel("X (mm)", fontsize=16)
ax.set_ylabel("Line-Integrated $n_e$ (cm$^{-2}$)", fontsize=16);

The unwrapped phase shift can also be recovered directly from the `Interferometer` class by running `evaluate` with `unwrapped = True`.

In [ ]:
size = np.array([[-1, 1], [-1, 1]]) * 1.2 * u.mm
bins = [300, 300]
hax, vax, phase = obj.evaluate(wprobe, 400, size=size, bins=bins, unwrapped=True)
hax = hax.to(u.mm).value
vax = vax.to(u.mm).value

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot = ax.pcolormesh(hax, vax, phase.T / np.pi, cmap="binary", shading="auto")
ax.set_aspect("equal")
ax.set_xlabel("X (mm)", fontsize=16)
ax.set_ylabel("Z (mm)", fontsize=16)

cb = fig.colorbar(plot, ax=ax, orientation="vertical", pad=0.03, shrink=0.7)
cb.ax.set_ylabel("Phase Shift / $\pi$ (unwrapped)", fontsize=16);

As with any LineIntegratedDiagnostic, `Interferometer` is capable of line-integrating through grids at arbitrary angles as set by the provided source and detector vectors. The following example integrates through the test cylinder at an angle of $45^\circ$ to the cylinder axis.

In [ ]:
source = (0 * u.mm, -5 * u.mm, -5 * u.mm)
detector = (0 * u.mm, 5 * u.mm, 5 * u.mm)
obj = Interferometer(grid, source, detector, verbose=False)

size = np.array([[-1, 1], [-1, 1]]) * 1.5 * u.mm
bins = [300, 300]
hax, vax, phase = obj.evaluate(wprobe, 400, size=size, bins=bins, unwrapped=False)
hax = hax.to(u.mm).value
vax = vax.to(u.mm).value

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot = ax.pcolormesh(hax, vax, phase.T / np.pi, cmap="binary", shading="auto")
ax.set_aspect("equal")
ax.set_xlabel("X (mm)", fontsize=16)
ax.set_ylabel("Z (mm)", fontsize=16)

cb = fig.colorbar(plot, ax=ax, orientation="vertical", pad=0.03, shrink=0.7)
cb.ax.set_ylabel("Phase Shift / $\pi$", fontsize=16);

## Creating a Custom Diagnostic Class

[LineIntegratedDiagnostic]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic.rst#plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic

A [LineIntegratedDiagnostic] synthetic diagnostic for interferometry ([Interferometer]) is already implemented in PlasmaPy. However, in this notebook we will re-create this class in order to demonstrate how a custom subclass of [LineIntegratedDiagnostic] can be used to create a new synthetic diagnostic.

In order to predict the phase shift through a plasma, we therefore only need to know the density line-integrated along the probe beam axis. This can be accomplished by defining a [LineIntegratedDiagnostic] with an appropriate `_integrand()` method as follows.

In [ ]:
class ExampleInterferometer(LineIntegratedDiagnostic):
    def _integrand(self, pts):
        # Reshape the pts array from grid shape (nx, ny, nz, 3) to a list
        # of points (nx*ny*nz, 3) as required by the grids interpolators
        nx, ny, nz, ndim = pts.shape
        pts = np.reshape(pts, (nx * ny * nz, ndim))

        integrand = self.grid.volume_averaged_interpolator(pts, "n_e")

        # Reshape the integrands from (nx*ny*nz) to (nx, ny, nz)
        integrand = np.reshape(integrand, (nx, ny, nz))

        return integrand

    def evaluate(
        self,
        probe_freq: u.Hz,
        num,
        size=np.array([[-1, 1], [-1, 1]]) * u.cm,
        bins=[50, 50],
        collimated=True,
        unwrapped=False,
    ):

        n_c = (
            (const.eps0.si * const.m_e / const.e.si ** 2)
            * (2 * np.pi) ** 2
            * probe_freq ** 2
        ).to(u.cm ** -3)

        hax, vax, int_ne = self._line_integral(
            size=size, bins=bins, collimated=collimated, num=num
        )

        phase_shift = (-np.pi * probe_freq / (const.c.si * n_c)) * int_ne
        phase_shift = phase_shift.to(u.dimensionless_unscaled).value

        if not unwrapped:
            phase_shift = (phase_shift + np.pi) % (2 * np.pi) - np.pi

        return hax, vax, phase_shift

[LineIntegratedDiagnostic]: ../../api/plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic.rst#plasmapy.diagnostics.path_integrated_diagnostic.LineIntegratedDiagnostic

In this case, the `_integrand()` method will return the density at each of the integration points. The `evaluate()` method is a wrapper around the `_line_integral()` method of [LineIntegratedDiagnostic] that calculates the total phase shift from the line-integrated density. If the `unwrapped` keyword is not set, then the function takes the modulus with respect to $2 \pi$ to estimate where jumps in the phase will occur.

In order to demonstrate the use of this ExampleInterferometer class, we will create an interferogram from an example grid which contains a sphere of constant density.

In [ ]:
axis = np.linspace(-2, 2, num=200) * u.mm
xarr, yarr, zarr = np.meshgrid(axis, axis, axis, indexing="ij")
r = np.sqrt(xarr ** 2 + yarr ** 2 + zarr ** 2)
n_e = np.where(r < 1 * u.mm, 1, 0) * 3e19 / u.cm ** 3
grid = CartesianGrid(xarr, yarr, zarr)
grid.add_quantities(n_e=n_e)

fig, ax = plt.subplots(figsize=(6, 6))
ax.pcolormesh(
    axis.to(u.mm).value, axis.to(u.mm).value, grid["n_e"][:, 100, :], shading="auto"
)
ax.set_xlabel("X (mm)")
ax.set_ylabel("Z (mm)")
ax.set_aspect("equal")

Now we can initialize the `ExampleInterferometer` class by also defining the locations of the source and detector

In [ ]:
source = (0 * u.mm, -5 * u.mm, 0 * u.mm)
detector = (0 * u.mm, 5 * u.mm, 0 * u.mm)
obj = Interferometer(grid, source, detector, verbose=False)

Finally we will specify the size and resolution of the detector plane when calling the `evaluate()` method to create the synthetic interferogram.

In [ ]:
size = np.array([[-1, 1], [-1, 1]]) * 1.2 * u.mm
bins = [300, 300]

hax, vax, phase = obj.evaluate(1.14e15 * u.Hz, 400, size=size, bins=bins)
hax = hax.to(u.mm).value
vax = vax.to(u.mm).value

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot = ax.pcolormesh(hax, vax, phase.T / np.pi, cmap="binary", shading="auto")
ax.set_aspect("equal")
ax.set_xlabel("X (mm)", fontsize=16)
ax.set_ylabel("Z (mm)", fontsize=16)

cb = fig.colorbar(plot, ax=ax, orientation="vertical", pad=0.03, shrink=0.7)
cb.ax.set_ylabel("Phase Shift / $\pi$", fontsize=16);